In [ ]:
import numpy as np 
import pandas as pd
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [ ]:
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([(

In [ ]:
lines=pd.read_csv("Hindi_English_Truncated_Corpus.csv",dtype={'english_sentence': object,'hindi_sentence':object})

In [ ]:
lines

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
...,...,...,...
127602,indic2012,Examples of art deco construction can be found...,आर्ट डेको शैली के निर्माण मैरीन ड्राइव और ओवल ...
127603,ted,and put it in our cheeks.,और अपने गालों में डाल लेते हैं।
127604,tides,"As for the other derivatives of sulphur , the ...","जहां तक गंधक के अन्य उत्पादों का प्रश्न है , द..."
127605,tides,its complicated functioning is defined thus in...,Zरचना-प्रकिया को उसने एक पहेली में यों बांधा है .


In [ ]:
lines=lines[~pd.isnull(lines['english_sentence'])]

In [ ]:
lines.drop_duplicates(inplace=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(lines['english_sentence'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
# https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stop_words) 
    preprocessed_reviews.append(sentance.strip())

100%|██████████| 124827/124827 [00:03<00:00, 41076.28it/s]


In [ ]:
lines['English_cleanedText']=preprocessed_reviews

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
exclude = set(string.punctuation)
remove_digits = str.maketrans('', '', digits)

In [ ]:
h_sentance=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))
h_sentance=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
h_sentance=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))
h_sentance = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))
h_sentance=lines['hindi_sentence'].apply(lambda x: x.strip())
h_sentance=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))
  

In [ ]:
lines['Hindi_cleanedText']=h_sentance

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
english=lines['English_cleanedText']
english_70k=english[:1000]
hindi=lines['Hindi_cleanedText']
hindi_70k=hindi[:1000]

In [ ]:
import collections

english_words_counter = collections.Counter([word for sentence in english_70k for word in sentence.split()])
hindi_words_counter = collections.Counter([word for sentence in hindi_70k for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_70k for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} hindi words.'.format(len([word for sentence in hindi_70k for word in sentence.split()])))
print('{} unique hindi words.'.format(len(hindi_words_counter)))
print('10 Most common words in the hindi dataset:')
print('"' + '" "'.join(list(zip(*hindi_words_counter.most_common(10)))[0]) + '"')

8252 English words.
4275 unique English words.
10 Most common words in the English dataset:
"one" "india" "also" "world" "people" "time" "made" "first" "government" "like"

19079 hindi words.
5546 unique hindi words.
10 Most common words in the hindi dataset:
"के" "में" "है" "की" "." "और" "से" "का" "को" ","


In [ ]:
import project_tests as tests

In [ ]:
def test_encdec_model(encdec_model):
    input_shape = (1000, 220, 1)
    output_sequence_length = 220
    english_vocab_size = 4275
    hindi_vocab_size = 5324

    model = encdec_model(input_shape, output_sequence_length, english_vocab_size, hindi_vocab_size)
    _test_model(model, input_shape, output_sequence_length, hindi_vocab_size)

In [ ]:
def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer




In [ ]:
import collections

import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy,binary_crossentropy

In [ ]:
def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

tests.test_tokenize(tokenize)

In [ ]:
def pad(x, length=None):
   
    return pad_sequences(x, maxlen=length, padding='post')

tests.test_pad(pad)


test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [ ]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_hindi_sentences, english_tokenizer, hindi_tokenizer =\
    preprocess(english_70k, hindi_70k)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_hindi_sequence_length = preproc_hindi_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
hindi_vocab_size = len(hindi_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max Hindi sentence length:", max_hindi_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("Hindi vocabulary size:", hindi_vocab_size)

Data Preprocessed
Max English sentence length: 71
Max Hindi sentence length: 220
English vocabulary size: 4275
Hindi vocabulary size: 5324


In [ ]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [ ]:
tmp_x.shape

(1000, 220, 1)

In [ ]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, hindi_vocab_size):
   
    learning_rate = 0.005
    
        
    model = Sequential()
    # Encoder
    model.add(GRU(64, input_shape=input_shape[1:], go_backwards=True))
    model.add(RepeatVector(output_sequence_length))
    # Decoder
    model.add(GRU(98, return_sequences=True))
    model.add(TimeDistributed(Dense(156, activation='relu')))
    model.add(Dropout(0.2))
    model.add(TimeDistributed(Dense(hindi_vocab_size, activation='softmax')))

    
    model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(learning_rate),metrics=['accuracy'])
                  
                  
    
    return model

tests.test_encdec_model(encdec_model)


tmp_x = pad(preproc_english_sentences, preproc_hindi_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_hindi_sentences.shape[-2], 1))


encdec_rnn_model = encdec_model(tmp_x.shape,preproc_hindi_sentences.shape[1],len(english_tokenizer.word_index)+1,len(hindi_tokenizer.word_index)+1)

encdec_rnn_model.summary()

encdec_rnn_model.fit(tmp_x, preproc_hindi_sentences, batch_size=150, epochs=10, validation_split=0.2)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_7 (GRU)                  (None, 56)                9744      
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 220, 56)           0         
_________________________________________________________________
gru_8 (GRU)                  (None, 220, 128)          71040     
_________________________________________________________________
time_distributed_7 (TimeDist (None, 220, 256)          33024     
_________________________________________________________________
dropout_4 (Dropout)          (None, 220, 256)          0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, 220, 5325)         1368525   
Total params: 1,482,333
Trainable params: 1,482,333
Non-trainable params: 0
____________________________________________

In [ ]:
print("Prediction:")
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], hindi_tokenizer))

print("\nCorrect Translation:")
print(hindi_70k[:1])

print("\nOriginal text:")
print(english_70k[:1])

Prediction:
<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD